In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-283r7u47/unsloth_47b63390626c4a188896790bf5c9a663
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-283r7u47/unsloth_47b63390626c4a188896790bf5c9a663
  Resolved https://github.com/unslothai/unsloth.git to commit 52bc19d1fa4cd3557b785127fd68b5f4d1c34347
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.7 MB/s eta 0:00:00
   ━━

In [ ]:
from unsloth import FastLanguageModel

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/gemma-7b-it-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Ankitnau25/govtbot-llama3.1",
    # model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1381: UserWarning: Current model requires 109052704.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

In [ ]:
model.generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "max_length": 131072,
  "pad_token_id": 128004,
  "temperature": 0.6,
  "top_p": 0.9
}

In [ ]:
import re
def predict (inp_text):
      messages = [
          {"from": "human", "value": f"{inp_text}"},
      ]
      inputs = tokenizer.apply_chat_template(
          messages,
          tokenize = True,
          add_generation_prompt = True, # Must add for generation
          return_tensors = "pt",
      ).to("cuda")
      model.generation_config.pad_token_id = tokenizer.pad_token_id
      outputs = model.generate(input_ids = inputs, use_cache = True ,temperature = 0.1)
      result = tokenizer.batch_decode(outputs)
      # print(result)
      return filter_user_assistant_msgs(result[0])

def filter_user_assistant_msgs(text):
  # Match the first user message
  # print(text)
  msg_pattern = r".*>assistant\n(.*?)<\|im_end\|>"
  match = re.match(msg_pattern, text, re.DOTALL)
  if match:
    message = match.group(1).strip()
  else:
    message = text
  return message

In [ ]:
 predict(" यदि यूएएन को आधार के साथ नहीं जोड़ा गया है, तो लॉक डाउन की अवधि के दौरान वैधीकरण/ सींडिग किस प्रकार संभव है?")

'यदि यूएएन को आधार के साथ नहीं जोड़ा गया है, तो लॉकडाउन की अवधि के दौरान वैधीकरण/सींडिग किस प्रकार संभव है?'

In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2e71926d453a6ad4ba39e6d0f87f55386e39355b921e06eb02620782192a7da4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score
from nltk.translate.meteor_score import meteor_score
import numpy as np
from nltk.tokenize import word_tokenize

nltk.download('wordnet')

# Ensure you have the required libraries installed
# pip install nltk rouge-score

# Dummy data for demonstration
ground_truth = [
    "You are requested to contact your bank..",
    "Aadhaar letter is laminated paper based document issued to the residents after enrollment and update. Aadhaar Card is a durable and easy to carry PVC card with security features. All forms of Aadhaar (eAadhaar, mAadhaar, Aadhaar letter, Aadhaar card) are equally valid. The resident has the choice to use any of these forms of Aadhaar issued by UIDAI. The ‘’Order Aadhaar PVC Card” service is available online on demand for the residents similar to online “Order Aadhaar Reprint” Service.",
    "In Cost to Company (CTC) model, if Rs.10000/- is monthly EPF wages, in CTC Model the employee gets Rs.200/- more directly from employer as employer’s EPF/EPS contribution is reduced and Rs.200/- less is deducted from his/her wages."
]

generated_responses = [
    "No, a fraudster cannot withdraw money from your Aadhaar linked bank account even if he knows your Aadhaar number or has your Aadhaar card.",
    "The Aadhaar PVC Card is a plastic card that contains the same information as the paper-based laminated Aadhaar Letter",
    "Yes, the reduced rate of contributions from 12% to 10% will still benefit you"

]

# Tokenize text
def tokenize(text):
    return word_tokenize(text.lower())

# Evaluation Functions
def evaluate_bleu(reference, candidate):
    # Smoothing helps in cases with short responses
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference], candidate, smoothing_function=smoothie)

def evaluate_rouge(reference, candidate):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return scores

def evaluate_meteor(reference, candidate):
    return meteor_score([reference], candidate)

def evaluate_accuracy(ground_truth, generated_responses):
    # This works if responses are expected to match exactly (e.g., classification tasks)
    correct = [gt == resp for gt, resp in zip(ground_truth, generated_responses)]
    return accuracy_score(ground_truth, generated_responses)

# Aggregate and display scores
def evaluate_responses(ground_truth, generated_responses):
    bleu_scores = []
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []
    meteor_scores = []

    for gt, resp in zip(ground_truth, generated_responses):
        gt_tokens = tokenize(gt)
        resp_tokens = tokenize(resp)

        bleu_scores.append(evaluate_bleu(gt, resp))

        rouge_scores = evaluate_rouge(gt, resp)
        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

        meteor_scores.append(evaluate_meteor(gt_tokens, resp_tokens))

    print("Average BLEU Score:", np.mean(bleu_scores))
    print("Average ROUGE-1 Score:", np.mean(rouge1_scores))
    print("Average ROUGE-2 Score:", np.mean(rouge2_scores))
    print("Average ROUGE-L Score:", np.mean(rougeL_scores))
    print("Average METEOR Score:", np.mean(meteor_scores))

    # For exact match scenarios:
    accuracy = evaluate_accuracy(ground_truth, generated_responses)
    print("Accuracy:", accuracy)

# Run evaluation
evaluate_responses(ground_truth, generated_responses)


[nltk_data] Downloading package wordnet to /root/nltk_data...


Average BLEU Score: 0.3502192388696825
Average ROUGE-1 Score: 0.3908869395711501
Average ROUGE-2 Score: 0.21199865183687228
Average ROUGE-L Score: 0.27860623781676414
Average METEOR Score: 0.2808854128888623
Accuracy: 0.0


In [1]:
git --version


NameError: name 'git' is not defined